# Sign Language Detection

This notebook demonstrates the usage of our sign language detection system.

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

from src.data import VideoDataset, create_dataloaders
from src.models import SignLanguageCNNLSTM, VideoTransformer, CNNLSTMConfig, TransformerConfig
from src.training import Trainer, TrainerConfig
from src.visualization import RealTimeInference
from src.config import PROCESSED_DIR, DATA_CONFIG

%matplotlib inline

2025-02-17 10:01:39.621579: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739768499.867545    8638 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739768499.936136    8638 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-17 10:01:40.462946: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Loading and Preprocessing

In [2]:
# Load preprocessed data
preprocessed_path = PROCESSED_DIR / 'preprocessing_results.json'
with open(preprocessed_path, 'r') as f:
    video_data = json.load(f)

print("Initial data load:")
print(f"Total videos: {len(video_data)}")

# Filter out failed preprocessing entries and verify data structure
video_data = [v for v in video_data if v.get('success', False)]

print(f"\nAfter filtering failed entries:")
print(f"Remaining videos: {len(video_data)}")

# Verify data structure
print("\nVerifying data structure...")
required_fields = ['gloss', 'frame_paths', 'bbox']
missing_fields = []
for field in required_fields:
    if not all(field in v for v in video_data):
        missing_fields.append(field)

if missing_fields:
    print(f"Warning: Missing required fields: {missing_fields}")
    print("\nSample entry:")
    print(json.dumps(video_data[0], indent=2))
else:
    print("All required fields present")

# Create class mapping
unique_glosses = sorted(list({v['gloss'] for v in video_data}))
class_mapping = {gloss: idx for idx, gloss in enumerate(unique_glosses)}

print(f"\nNumber of unique classes: {len(class_mapping)}")

# Create data loaders with debugging
train_split = DATA_CONFIG['train_split']
val_split = DATA_CONFIG['val_split']
test_split = 1.0 - train_split - val_split

print(f"\nData split proportions:")
print(f"Train: {train_split:.2%}")
print(f"Validation: {val_split:.2%}")
print(f"Test: {test_split:.2%}")

# Expected split sizes
total_samples = len(video_data)
expected_train = int(total_samples * train_split)
expected_val = int(total_samples * val_split)
expected_test = total_samples - expected_train - expected_val

print(f"\nExpected split sizes:")
print(f"Train: {expected_train}")
print(f"Validation: {expected_val}")
print(f"Test: {expected_test}")

# Create dataloaders
train_loader, val_loader, test_loader = create_dataloaders(
    video_data=video_data,
    class_mapping=class_mapping,
    batch_size=DATA_CONFIG['batch_size']
)

print(f"\nActual split sizes:")
print(f"Train: {len(train_loader.dataset)}")
print(f"Validation: {len(val_loader.dataset)}")
print(f"Test: {len(test_loader.dataset)}")

# Verify dataloaders
print("\nVerifying dataloaders...")
try:
    print("Checking training loader:")
    train_batch = next(iter(train_loader))
    print(f"Training batch shapes:")
    print(f"Frames: {train_batch[0].shape}")
    print(f"Labels: {train_batch[1][0].shape}")
    print(f"Bboxes: {train_batch[1][1].shape}")
    
    print("\nChecking validation loader:")
    val_batch = next(iter(val_loader))
    print(f"Validation batch shapes:")
    print(f"Frames: {val_batch[0].shape}")
    print(f"Labels: {val_batch[1][0].shape}")
    print(f"Bboxes: {val_batch[1][1].shape}")
except Exception as e:
    print(f"Error loading batch: {str(e)}")

Initial data load:
Total videos: 6

After filtering failed entries:
Remaining videos: 0

Verifying data structure...
All required fields present

Number of unique classes: 0

Data split proportions:
Train: 70.00%
Validation: 15.00%
Test: 15.00%

Expected split sizes:
Train: 0
Validation: 0
Test: 0


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
# Training configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

trainer_config = TrainerConfig(
    num_epochs=50,
    learning_rate=1e-4,
    use_wandb=False
)

# Initialize model
print("\nInitializing CNN-LSTM model...")
cnn_lstm_config = CNNLSTMConfig(num_classes=len(class_mapping))
cnn_lstm = SignLanguageCNNLSTM(cnn_lstm_config).to(device)

# Only proceed with training if validation data exists
if len(val_loader.dataset) > 0:
    trainer = Trainer(cnn_lstm, trainer_config)
    print("\nStarting CNN-LSTM training...")
    cnn_lstm_history = trainer.train(train_loader, val_loader)
else:
    print("\nError: No validation data available. Cannot proceed with training.")